# Apply RNAseg on spatial data 

In [2]:
import spatialdata as sd
from pathlib import Path
from rna2seg.dataset_zarr.patches import create_patch_rnaseg
from rna2seg.dataset_zarr.RNAsegDataset import RNAsegDataset
from rna2seg.models import RNA2seg
import albumentations as A
import cv2

/home/tom/anaconda3/envs/RNAseg/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/tom/anaconda3/envs/RNAseg/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warn(f"Failed to load image Python extension: {e}")


No module named 'vmunet'
VMUnet not loaded


In [3]:
from rna2seg.models import RNAEmbedding

# 1) Create dataset from a spatialdata

In [4]:
### load sdata and set path parameters 
merfish_zarr_path = "/media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr"
sdata = sd.read_zarr(merfish_zarr_path)

image_key = "staining_z3"
points_key = "transcripts"

patch_width = 1200
patch_overlap = 150



min_transcripts_per_patch = 0
folder_patch_rna_seg = Path(merfish_zarr_path) / ".rna_seg"

### create patch in the sdata and precompute transcipt.csv for each patch with sopa
create_patch_rnaseg(sdata,
                    image_key=image_key,
                    points_key=points_key,
                    patch_width=patch_width,
                    patch_overlap=patch_overlap,
                    min_transcripts_per_patch=min_transcripts_per_patch,
                    overwrite = True)
print(sdata)

/home/tom/anaconda3/envs/RNAseg/lib/python3.10/site-packages/spatialdata/_core/_elements.py:96: UserWarning: Key `sopa_patches_rna_seg_1200_150` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)
[INFO] (sopa.patches.patches) 81 patches were saved in sdata['sopa_patches_rna_seg_1200_150']


[########################################] | 100% Completed | 21.83 ss
SpatialData object, with associated Zarr store: /media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr
├── Images
│     └── 'staining_z3': DataTree[cyx] (5, 9000, 9000), (5, 4500, 4500), (5, 2250, 2250), (5, 1125, 1125), (5, 562, 562)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 9) (2D points)
└── Shapes
      ├── 'Cellbound1': GeoDataFrame shape: (3258, 1) (2D shapes)
      ├── 'Cellbound1_consistent_with_nuclei': GeoDataFrame shape: (1007, 1) (2D shapes)
      ├── 'Cellbound1_consistent_without_nuclei': GeoDataFrame shape: (2239, 1) (2D shapes)
      ├── 'DAPI': GeoDataFrame shape: (2377, 1) (2D shapes)
      ├── 'DAPI_consistent_in_cell': GeoDataFrame shape: (1007, 1) (2D shapes)
      ├── 'DAPI_consistent_not_in_cell': GeoDataFrame shape: (1370, 1) (2D shapes)
      ├── 'sopa_patches_rna_seg_1200_50': GeoDataFrame shape: (64, 3) (2D shapes)
      └── 'sopa

In [5]:
transform_resize  = A.Compose([
 A.Resize(width=512, height=512, interpolation=cv2.INTER_NEAREST),
])


dataset = RNAsegDataset(
sdata=sdata,
channels_dapi= ["DAPI"],
channels_cellbound=["Cellbound1"],
patch_width = 1200,
patch_overlap = 150,
gene_column="gene",
transform_resize = transform_resize,
 )

default shape_patch_key set to sopa_patches_rna_seg_1200_150
100%|██████████| 81/81 [00:02<00:00, 35.16it/s]
Number of valid patches: 64


# 2) run RNAseg

In [7]:

pretrained_model = "/media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/small_merfish_for_test.zarr/model/best.pt"
device = "cpu"
rnaseg = RNA2seg(device,
                net='unet',
                flow_threshold = 0.9,
                cellbound_flow_threshold = 0.4,
                pretrained_model = pretrained_model,
                )

initiaisation of CPnet
Initiaisation of ChannelInvariantNet
Loading weights from /media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/small_merfish_for_test.zarr/model/best.pt


# 3) Add new segmentation to the input spatial data

In [9]:
from tqdm import tqdm
path_temp_save=dataset.st_segmentation.sdata.path / '.rna2seg_seg'
for i in tqdm(list(range(len(dataset)))):
    input_dict = dataset[i]
    rnaseg.run(
        path_temp_save=path_temp_save,
        input_dict=input_dict
    )



[INFO] (sopa.segmentation.shapes) Percentage of non-geometrized cells: 0.00% (usually due to segmentation artefacts)
  0%|          | 0/64 [00:03<?, ?it/s]

90 cells detected


TypeError: unsupported operand type(s) for /: 'PosixPath' and 'int'

# 4) Add merge the 

tensor([[[ 0.3373,  0.3294,  0.3038,  ...,  0.1389,  0.1610,  0.1832],
         [ 0.3062,  0.2798,  0.2649,  ...,  0.1546,  0.1919,  0.2171],
         [ 0.2690,  0.2605,  0.2524,  ...,  0.1821,  0.2148,  0.2450],
         ...,
         [ 0.1838,  0.1742,  0.1682,  ..., -0.0051, -0.0066, -0.0033],
         [ 0.2102,  0.1960,  0.1944,  ..., -0.0060, -0.0096, -0.0100],
         [ 0.2275,  0.2186,  0.2045,  ..., -0.0099, -0.0047, -0.0136]]])